In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from tqdm import tqdm  # I love this package, it makes me feel like a real coder - MV

# Mark Viti \& Marcus Lisman

## Problem 1

### 1
**Problem:**
For $Y = X\beta + U$ where we observe $Y, \tilde{X}, \u{X}$ such that $\tilde{X} = X + V^1$ and $\u{X} = X + V^2$. Moreover, we have that for $j = {1, 2}$, $\mathbb{E}\left[V^j\right] = 0, \mathbb{E}[XV^j] = 0, \mathbb{E}[V^1V^2] = 0$, $\mathbb{E}\left[UV^j\right] = 0$, $\mathbb{E}\left[V^1V^2\right]$. Verify the relevance and exogeneity of $\u{X}$ as an instrument for $\tilde{X}$.

**Solution:**
Recall the definitions of relevance and exogeneity:
- Relevance: $\mathbb{E}\left[\u{X}\tilde{X}\right] \neq 0$
- Exogeneity: $\mathbb{E}\left[\u{X}U\right] = 0$

First, we verify relevance:
$\begin{align}
\mathbb{E}\left[\u{X}\tilde{X}\right] &= \mathbb{E}\left[\left(X + V^2\right)\left(X + V^1\right)\right] \\
&= \mathbb{E}\left[X^2 + XV^1 + XV^2 + V^1V^2\right] \\
&= \mathbb{E}\left[X^2\right] + \mathbb{E}\left[XV^1\right] + \mathbb{E}\left[XV^2\right] + \mathbb{E}\left[V^1V^2\right] \\
&= \mathbb{E}\left[X^2\right] + 0 + 0 + 0 \\
&= \mathbb{E}\left[X^2\right] \neq 0
\end{align}$
Now, someone asked on Ed if we can assume that $var{X} > 0$, which we can. This it is clear that $\mathbb{E}\left[\u{X}\tilde{X}\right] \neq 0$.

Next, we verify exogeneity:
$\begin{align}
\mathbb{E}\left[\u{X}U\right] &= \mathbb{E}\left[\left(X + V^2\right)U\right] \\
&= \mathbb{E}\left[XU + V^2U\right] \\
&= \mathbb{E}\left[XU\right] + \mathbb{E}\left[V^2U\right] \\
&= \mathbb{E}\left[XU\right] + 0 \\
&= 0
\end{align}$
Thus, $\u{X}$ is both relevant and exogenous as an instrument for $\tilde{X}$.
QED

### 2
**Problem:**
We have the following supply and demand equations:

$Q^s(P) = \alpha^s + P\beta^s + Z_\gamma + U^s$

$Q^d(P) = \alpha^d + P\beta^d + U^d$
- - - 
**(A)**: What is the expected sign of $\beta^s$, $\beta^d$, and $\beta^d - \beta^s$?

The expected sign of $\beta^s$ is positive, as the quantity supplied should increase with price. The expected sign of $\beta^d$ is negative, as the quantity demanded should decrease with price. Thus, the expected sign of $\beta^d - \beta^s$ is negative.

- - - 

**(B)**: Derive the equilibrium price. 

At equilibrium, $Q^s(P) = Q^d(P)$. Thus, we have:
$\begin{align}
\alpha^s + P\beta^s + Z_\gamma + U^s &= \alpha^d + P\beta^d + U^d \\
P\beta^s - P\beta^d &= \alpha^d - \alpha^s + U^s - U^d - Z_\gamma \\
P(\beta^s - \beta^d) &= \alpha^d - \alpha^s + U^s - U^d - Z_\gamma \\
P &= \frac{\alpha^d - \alpha^s + U^s - U^d - Z_\gamma}{\beta^s - \beta^d}
\end{align}$
We are going to call this price $P^*$.

- - - 

**(C)**: Show that the equilibrium price is endogenous in the model for demand.

We have that $P^* = \frac{\alpha^d - \alpha^s + U^s - U^d - Z_\gamma}{\beta^s - \beta^d}$. We can see that $P^*$ is endogenous in the model for demand because $U^d$ is a random variable that is not observed. Thus, $P^*$ is a function of $U^d$ and is endogenous in the model for demand.

- - -

**(D)**: Show that $Z$ is relevant and exogenous as an instrument for $P$ in the demand model. Assume that $\gamma \neq 0, \mathbb{E}\left[ZU^s\right] = \mathbb{E}\left[ZU^d\right] = \mathbb{E}\left[U^sU^d\right] = 0$ and $var(Z) > 0$.

To do so, we want to show that $\mathbb{E}\left[ZP\right] \neq 0$ and $\mathbb{E}\left[ZU^d\right] = 0$. Note that we are assuming that $\mathbb{E}\left[ZU^s\right] = 0$ and $\mathbb{E}\left[ZU^d\right] = 0$, so we already have exogeneity.

Thus, we need to show relevance. 
$\begin{align}
\mathbb{E}\left[ZP\right] &= \mathbb{E}\left[Z\frac{\alpha^d - \alpha^s + U^s - U^d - Z_\gamma}{\beta^s - \beta^d}\right] \\
&= \frac{\alpha^d - \alpha^s + \mathbb{E}\left[ZU^s\right] - \mathbb{E}\left[ZU^d\right] - \mathbb{E}\left[Z^2\gamma\right]}{\beta^s - \beta^d} \\
&= \frac{\alpha^d - \alpha^s - \mathbb{E}\left[Z^2\gamma\right]}{\beta^s - \beta^d} \neq 0
\end{align}$
Thus, $Z$ is relevant and exogenous as an instrument for $P$ in the demand model.

- - -

**(E)**: Is $Z$ relevant as an instrument for $P$ in the supply model?

No. $Z$ is in the model itself, so it is not relevant as an instrument for $P$ in the supply model.


## Problem 2

### 1

In [2]:
def generate_data(n: int, b1: float, b2: float, b3: float, b4: float, gamma1: float, gamma2: float):
    x2 = np.random.normal(0, 1, n)
    v = np.random.normal(0, 0.1, n)
    x1 = x2 + x2**2*gamma1 + x2**5*gamma2 + v
    u = np.random.normal(0, 1, n)
    y = b1 * x1 + b2 * x2 + x2**2*b3 + np.sin(x2)*b4 + u

    return (y, x1, x2)

### 2

In [3]:
import numpy as np
import statsmodels.api as sm

def estimate_coefficients(n, b1, b2):
    # Generate data as per the previous function
    x2 = np.random.normal(0, 1, n)
    v = np.random.normal(0, 0.1, n)
    x1 = x2 + v  # Since gamma1 = gamma2 = 0
    u = np.random.normal(0, 1, n)
    y = b1 * x1 + b2 * x2 + u  # Since beta3 = beta4 = 0

    # Unrestricted Regression (y ~ x1 + x2)
    X_unrestricted = sm.add_constant(np.column_stack((x1, x2)))
    unrestricted_model = sm.OLS(y, X_unrestricted).fit()
    b1_unrestricted = unrestricted_model.params[1]
    se_unrestricted = unrestricted_model.bse[1]

    # Restricted Regression (y ~ x1)
    X_restricted = sm.add_constant(x1)
    restricted_model = sm.OLS(y, X_restricted).fit()
    b1_restricted = restricted_model.params[1]
    se_restricted = restricted_model.bse[1]

    # Pretest Regression: Using AIC/BIC or p-value for model selection could be an approach
    # Here, the example will use AIC as a simple criterion
    if unrestricted_model.aic < restricted_model.aic:
        chosen_model = 'Unrestricted'
        b1_pretest = b1_unrestricted
        se_pretest = se_unrestricted
    else:
        chosen_model = 'Restricted'
        b1_pretest = b1_restricted
        se_pretest = se_restricted

    return {
        'Unrestricted': {'Coefficient': b1_unrestricted, 'Standard Error': se_unrestricted},
        'Restricted': {'Coefficient': b1_restricted, 'Standard Error': se_restricted},
        'Pretest': {'Chosen Model': chosen_model, 'Coefficient': b1_pretest, 'Standard Error': se_pretest}
    }


### 3

In [4]:

n_values = [10, 100, 1000, 10000]
S = 1000
b1 = 2
results = []

for n in n_values:
    b2 = 3 * np.sqrt(n)
    for s in tqdm(range(S)):  # tqdm is optional
        # Generate dataset
        y, x1, x2 = generate_data(n, b1, b2, 0, 0, 0, 0)  # Assuming gamma values are needed and set to 0

        # Compute estimators
        estimators = estimate_coefficients(n, b1, b2)
        
        results.append({
            'n': n,
            's': s,
            'Unrestricted Coefficient': estimators['Unrestricted']['Coefficient'],
            'Unrestricted SE': estimators['Unrestricted']['Standard Error'],
            'Restricted Coefficient': estimators['Restricted']['Coefficient'],
            'Restricted SE': estimators['Restricted']['Standard Error'],
            'Pretest Chosen Model': estimators['Pretest']['Chosen Model'],
            'Pretest Coefficient': estimators['Pretest']['Coefficient'],
            'Pretest SE': estimators['Pretest']['Standard Error']
        })

df_results = pd.DataFrame(results)


100%|██████████| 1000/1000 [00:08<00:00, 120.64it/s]


### 4

In [5]:
summary_stats = df_results.groupby('n').agg({
    'Unrestricted Coefficient': ['mean', 'var'],
    'Restricted Coefficient': ['mean', 'var'],
    'Pretest Coefficient': ['mean', 'var']
}).reset_index()

print(summary_stats)


       n Unrestricted Coefficient            Restricted Coefficient            \
                             mean        var                   mean       var   
0     10                 2.087807  17.786043              11.398245  0.249760   
1    100                 2.104309   1.037656              31.715019  0.101314   
2   1000                 2.004309   0.098573              95.936805  0.090079   
3  10000                 1.999758   0.009761             299.020228  0.089247   

  Pretest Coefficient             
                 mean        var  
0            2.524996  22.966214  
1            2.104309   1.037656  
2            2.004309   0.098573  
3            1.999758   0.009761  


In [6]:
true_model_counts = df_results.groupby('n')['Pretest Chosen Model'].value_counts(normalize=True).unstack().reset_index()

print(true_model_counts)


Pretest Chosen Model      n  Restricted  Unrestricted
0                        10       0.134         0.866
1                       100         NaN         1.000
2                      1000         NaN         1.000
3                     10000         NaN         1.000


In [7]:
def ci_coverage_and_length(df, true_coefficient):
    """
    Calculate the coverage and length of confidence intervals for each estimator type.
    """
    results = []
    for estimator in ['Unrestricted', 'Restricted', 'Pretest']:
        coefficient_col = f'{estimator} Coefficient'
        se_col = f'{estimator} SE'
        
        # Calculate CI bounds
        ci_lower = df[coefficient_col] - 1.96 * df[se_col]
        ci_upper = df[coefficient_col] + 1.96 * df[se_col]
        
        # Determine if true coefficient is within CI
        df[f'{estimator} Contains True'] = (ci_lower <= true_coefficient) & (ci_upper >= true_coefficient)
        
        df[f'{estimator} CI Length'] = ci_upper - ci_lower
        
        coverage_fraction = df[f'{estimator} Contains True'].mean()
        average_ci_length = df[f'{estimator} CI Length'].mean()
        
        results.append({
            'Model': estimator,
            'Coverage Fraction': coverage_fraction,
            'Average CI Length': average_ci_length
        })

    return pd.DataFrame(results)

true_coefficient_b1 = 2 
summary_ci = ci_coverage_and_length(df_results, true_coefficient_b1)

print(summary_ci)


          Model  Coverage Fraction  Average CI Length
0  Unrestricted              0.939           5.114588
1    Restricted              0.000           1.369599
2       Pretest              0.912           4.481197


### 5